### Задание 1
Напишите функцию, которая возвращает название валюты (поле ‘Name’) с максимальным значением курса с помощью сервиса https://www.cbr-xml-daily.ru/daily_json.js



In [1]:
import requests
class Rate:
    def __init__(self, format_='value'):        
        self.format = format_
    
    def exchange_rates(self):
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return r.json()['Valute']
    
    def make_format(self, currency):
        """
        Возвращает информацию о валюте currency в двух вариантах:
        - полная информация о валюте при self.format = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        
        Rate('value').make_format('EUR')
        79.4966
        """
        response = self.exchange_rates()
        
        if currency in response:
            if self.format == 'full':
                return response[currency]
            
            if self.format == 'value':
                return response[currency]['Value']
        
        return 'Error'
    
    def eur(self):
        """Возвращает курс евро на сегодня в формате self.format"""
        return self.make_format('EUR')
    
    def usd(self):
        """Возвращает курс доллара на сегодня в формате self.format"""
        return self.make_format('USD')
    
    def brl(self):
        """Возвращает курс бразильского реала на сегодня в формате self.format"""
        return self.make_format('BRL')    

    def max_currency(self):
        valutes = self.exchange_rates()
        max_cur = None
        max_val = None        
        for val in valutes.values():
            if max_val is None or max_val < val['Value']:
                max_cur = val['CharCode']
                max_val = val['Value']
            
        return valutes[max_cur]['Name']
    
        
ex = Rate()
ex.max_currency()

'СДР (специальные права заимствования)'

### Задание 2
Добавьте в класс Rate параметр diff (со значениями True или False), который в случае значения True в методах eur и usd будет возвращать не курс валюты, а изменение по сравнению в прошлым значением. Считайте, self.diff будет принимать значение True только при возврате значения курса. При отображении всей информации о валюте он не используется.



In [2]:
class Rate:
    def __init__(self, format_='value', diff_ = False):        
        self.format = format_
        self.diff   = diff_
    
    def exchange_rates(self):
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return r.json()['Valute']
    
    def make_format(self, currency, format_=None, diff_ = False):
        """
        Возвращает информацию о валюте currency в двух вариантах:
        - полная информация о валюте при self.format = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        
        Rate('value').make_format('EUR')
        79.4966
        """
        response = self.exchange_rates()
        
        if not format_:
            format_ = self.format            
        
        if currency in response:
            if format_ == 'full':
                return response[currency]
            
            if format_ == 'value':
                if diff_:
                    # полагаемся на то, что только в случае format==value diff м.б. True
                    return response[currency]['Value']-response[currency]['Previous']
                return response[currency]['Value']
        
        return 'Error'
    
       
    def eur(self):
        """Возвращает курс евро на сегодня в формате self.format"""        
        return self.make_format('EUR', diff_ = self.diff)
    
    def usd(self):
        """Возвращает курс доллара на сегодня в формате self.format"""
        return self.make_format('USD', diff_ = self.diff)
    
    def brl(self):
        """Возвращает курс бразильского реала на сегодня в формате self.format"""
        return self.make_format('BRL')    

    def max_currency(self):
        valutes = self.exchange_rates()
        max_cur = None
        max_val = None        
        for val in valutes.values():
            if max_val is None or max_val < val['Value']:
                max_cur = val['CharCode']
                max_val = val['Value']
            
        return valutes[max_cur]['Name']

rate = Rate()
drate= Rate(diff_=True)
print('USD value={} diff={}'.format(rate.usd(), drate.usd()))
print('EUR value={} diff={}'.format(rate.eur(), drate.eur()))
# на brl работать не будет
print('BRL value={} diff={}'.format(rate.brl(), drate.brl()))

USD value=71.585 diff=-0.012399999999999523
EUR value=83.7759 diff=0.6727999999999952
BRL value=13.679 diff=13.679


### Задание 3
Напишите класс Designer, который учитывает количество международных премий. Подсказки в коде занятия в разделе “Домашнее задание задача 3”.

In [3]:
class Employee:
    def __init__(self, name, seniority):
        self.name = name
        self.seniority = seniority
        
        self.grade = 1
    
    def grade_up(self):
        """Повышает уровень сотрудника"""
        self.grade += 1
    
    def publish_grade(self):
        """Публикация результатов аккредитации сотрудников"""
        print(self.name, self.grade)
        
class Developer(Employee):
    def __init__(self, name, seniority):
        super().__init__(name, seniority)
    
    def check_if_it_is_time_for_upgrade(self):
        # для каждой аккредитации увеличиваем счетчик на 1
        # пока считаем, что все разработчики проходят аккредитацию
        self.seniority += 1
        
        # условие повышения сотрудника из презентации
        if self.seniority % 5 == 0:
            self.grade_up()
        
        # публикация результатов
        return self.publish_grade()
    
# проверяем как работает система повышения сотрудников на примере отдела разработки

# разработчик Александр только что пришел в компанию
alex = Developer('Александр', 0)

for i in range(20):
    alex.check_if_it_is_time_for_upgrade()
    
irina = Developer('Ирина', 5)

for i in range(20):
    irina.check_if_it_is_time_for_upgrade()

Александр 1
Александр 1
Александр 1
Александр 1
Александр 2
Александр 2
Александр 2
Александр 2
Александр 2
Александр 3
Александр 3
Александр 3
Александр 3
Александр 3
Александр 4
Александр 4
Александр 4
Александр 4
Александр 4
Александр 5
Ирина 1
Ирина 1
Ирина 1
Ирина 1
Ирина 2
Ирина 2
Ирина 2
Ирина 2
Ирина 2
Ирина 3
Ирина 3
Ирина 3
Ирина 3
Ирина 3
Ирина 4
Ирина 4
Ирина 4
Ирина 4
Ирина 4
Ирина 5


Напишите класс Designer, который учитывает количество международных премий для дизайнеров (из презентации: "Повышение на 1 грейд за каждые 7 баллов. Получение международной премии – это +2 балла"). Считайте, что при выходе на работу сотрудник уже имеет две премии и их количество не меняется со стажем (конечно если хотите это можно вручную менять). Выполните проверку для 20 аккредитаций дизайнера Елены.

Для проверки используйте код:

In [4]:
# designer не developer, всё таки
class Designer(Employee): 
    
    def __init__(self, name, seniority, awards):
        super().__init__(name, seniority)
        self.awards = awards
        
    def seniority_score(self):
        """
        Расчитать и вернуть "score" для сотрудника
        Чуть лучше бы было это в базовый класс вынести
        и туда же "необходимый левел" для повышения грейда.        
        """
        return self.seniority+self.awards * 2
    
    def check_if_it_is_time_for_upgrade(self):
        self.seniority += 1
        
        if self.seniority_score() % 7 == 0:
            self.grade_up()
        
        return self.publish_grade()

elena = Designer('Елена', seniority=0, awards=2)
for i in range(20):    
    elena.check_if_it_is_time_for_upgrade()
    print('Seniority score: ', elena.seniority_score())

Елена 1
Seniority score:  5
Елена 1
Seniority score:  6
Елена 2
Seniority score:  7
Елена 2
Seniority score:  8
Елена 2
Seniority score:  9
Елена 2
Seniority score:  10
Елена 2
Seniority score:  11
Елена 2
Seniority score:  12
Елена 2
Seniority score:  13
Елена 3
Seniority score:  14
Елена 3
Seniority score:  15
Елена 3
Seniority score:  16
Елена 3
Seniority score:  17
Елена 3
Seniority score:  18
Елена 3
Seniority score:  19
Елена 3
Seniority score:  20
Елена 4
Seniority score:  21
Елена 4
Seniority score:  22
Елена 4
Seniority score:  23
Елена 4
Seniority score:  24
